##### Configurar ambiente virtual (Opcional)
Para criar um ambiente virtual, abra o terminal e execute os comandos abaixo:

```bash
python -m venv .venv # Será criado uma pasta chamada .venv, que será o ambiente virtual

.venv\Scripts\activate # Realiza a ativação do ambiente virtual (Windows)
```

##### Para instalar as dependências necessárias rodar o comando abaixo

In [1]:
%pip install -r credenciais/requirements.txt

  Obtaining dependency information for requests==2.31.0 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/b6/7c/8debebb4f90174074b827c63242c23851bdf00a532489fba57fef3416e40/charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.5 kB ? eta -:--:--
     ---------------------------------------- 61.5/61.5 kB 3.2 MB/s eta 0:00:00
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/96/94/c31f58c7a7f470d5665935262ebd7455c7e4c7782eb525658d3dbf4b9403/urllib3-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf7c3d9/certifi-2023.7.22-py3-


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


##### Realização de imports e carregamento de variáveis ambiente

##### **Importante**: Preencha o arquivo .env e em seguida recarregue o ambiente
- No VS Code, isso pode ser feito apertando CTRL + Shift + P (Windows / Linux) / CMD + SHIFT + P (Mac) e utilizar a opção (Reload Window / Recarregar janela)

In [2]:
from dotenv import load_dotenv
from datetime import datetime

import sys
import os
import requests
import json
import pandas as pd

env_file_path = os.path.join('credenciais', '.env')

load_dotenv(dotenv_path=env_file_path)

True

##### Tratamento de erros

##### O tratamento de erros é realizado de acordo com a variável de ambiente **RUNNING_ENV**

In [3]:
class AppError(Exception):
  def __init__(self, message):
    self.message = message

def handleErrors(error_message):
  running_environment = os.environ['RUNNING_ENV']
  if running_environment.lower() == 'notebook':
    raise AppError(error_message)
  else:
    print(error_message)
    sys.exit(1)

##### Função que realiza consulta na API

Link da documentação: https://currencybeacon.com/api-documentation

Descrição e uso das variáveis internas da função:
- base: Moeda que é usada para classificação - Os valores para a variável base. (linha 3) estão disponíveis em: https://currencybeacon.com/supported-currencies
- start_date: Data inicial para busca de dados históricos.
- end_date: Data final para busca de dados históricos. Atualmente está sendo preenchida com a data de execução do script
- API_KEY: Disponibilizada após a criação de conta na API e utilizada na consulta aos dados.

In [4]:
def get_api_data():
  API_BASE_URL = 'https://api.currencybeacon.com/v1'
  base = 'BRL'
  start_date = '2023-11-01' # Data de início para dados históricos
  end_date =   str(datetime.now().date()) # Data de fim para dados históricos
  API_KEY = os.environ['API_KEY']
  response = None

  request_url = f'{API_BASE_URL}/timeseries?base={base}&start_date={start_date}&end_date={end_date}&api_key={API_KEY}'
  response = requests.get(request_url)
  
  if not response.status_code == 200:
    handleErrors('Falha ao buscar dados da API')
  return response.json()

##### Cria um arquivo **data.json** dentro da pasta **dados** e salva os dados da resposta da API (em caso de sucesso).

In [5]:
def save_json_file(data):
  try:
    data_to_save = data['response']

    with open('dados/data.json', 'w', encoding='utf-8') as file:
      json.dump(data_to_save, file, ensure_ascii=False, indent=2)
  except:
    handleErrors('Falha ao salvar arquivo json')

##### Dado um arquivo JSON, a função irá ler o arquivo e retornar os dados.

In [6]:
def get_data_from_json_file(json_filename):
  try:
    with open(json_filename, 'r', encoding='utf-8') as json_file:
      json_data = json.load(json_file)
  
    return json_data
  except Exception:
    handleErrors(f'Falha ao ler dados de arquivo {json_filename}')

A função realiza os seguintes passos:<br>
1 - Verifica se o arquivo **data.json** existe.<br>
2 - Caso o arquivo não exista, será realizada uma chamada para API e a criação do arquivo **data.json**.<br>
3- Caso o arquivo exista, a função irá ler os dados do arquivo ao invés de buscar na API, para assim economizar chamadas realizadas a API, dado que existe um limite. 

In [7]:
data_file = f'{os.getcwd()}\\dados\\data.json'

data_file_exists = os.path.exists(data_file)
data = None

if data_file_exists is False:
  data = get_api_data()
  save_json_file(data)
  data = get_data_from_json_file('dados\\data.json')
else:
  data = get_data_from_json_file('dados\\data.json')

Início do tratamento dos dados. Abaixo são realizadas as seguintes etapas:<br>
1 - Geração de um dataframe com os dados obtidos.<br>
2 - Converter a coluna index em uma coluna comum com as moedas.<br>
3 - Fazer um pivot, transformando as colunas de datas em linhas.<br>

In [8]:
df = pd.DataFrame(data)

df.index.name = 'moeda'
df = df.reset_index()

df_cotacoes = pd.melt(df, id_vars=['moeda'], var_name='data', value_name='valor')

Leitura do arquivo JSON que contém as descrições das moedas.

In [9]:
currency_descriptions = get_data_from_json_file('dados/data_descricoes.json')

Manipulação dos dados das descrições das moedas. São realizadas as seguintes etapas:<br>
1 - Carregar as informações das descrições das moedas, a partir dos dados lidos do JSON contendo as descrições.<br>
2 - Fazer o pivot das colunas.

In [10]:
df_moedas = pd.DataFrame(currency_descriptions,index=[0])

df_moedas_descricoes = df_moedas.melt(var_name='moeda', value_name='descricao')

Realização do merge (junção) entre as tabelas de cotações e descrições, para adicionar a coluna de descrições à tabela de cotações.<br>
Substituição de (.) por (,) para evitar problemas de exibição de casas decimais

In [14]:
df_resultado = pd.merge(df_cotacoes, df_moedas_descricoes, on='moeda', how='left')
df_resultado['valor'] = df_resultado['valor'].astype(str).str.replace('.', ',')

Exportando os dados gerados e tratados para um arquivo csv

In [12]:
try:
  df_resultado.to_csv('dados\\df_cotacoes.csv', index=False)
except Exception:
  handleErrors('Falha ao gerar arquivo csv')